In [1]:
from pygments.lexers import configs

from models.compositors import transformer_factory
from models.image_encoders import image_encoder_factory
from models.text_encoders import text_encoder_factory
from models.augmenter import augmenter_factory
from utils.mixins import GradientControlDataParallel
import torch

def create_models(configs, vocabulary):
    text_encoder, text_fc = text_encoder_factory(vocabulary, config=configs)
    lower_img_encoder, upper_img_encoder = image_encoder_factory(config=configs)

    layer_shapes = lower_img_encoder.layer_shapes()
    compositors = transformer_factory({'layer4': layer_shapes['layer4'],
                                       'image_feature_size': upper_img_encoder.feature_size,
                                       'text_feature_size': text_encoder.feature_size}, configs=configs)
    augmenter = augmenter_factory(config=configs)

    models = {
        'text_encoder': text_encoder,
        'lower_image_encoder': lower_img_encoder,
        'upper_image_encoder': upper_img_encoder,
    }
    if text_fc != None:
        models['text_fc'] = text_fc
    if augmenter != None:
        models['augmenter'] = augmenter
    models.update(compositors)

    if configs['num_gpu'] > 1:
        for name, model in models.items():
            models[name] = GradientControlDataParallel(model.cuda())

    return models


In [3]:
from data import DEFAULT_VOCAB_PATHS, create_dataloaders
from evaluators import get_evaluator_cls
from language import vocabulary_factory
from loggers.file_loggers import BestModelTracker
from loggers.wandb_loggers import WandbSimplePrinter, WandbSummaryPrinter
from losses import loss_factory
from models import create_models
from optimizers import create_optimizers, create_lr_schedulers
from options import get_experiment_config
from set_up import setup_experiment
from trainers import get_trainer_cls
from transforms import image_transform_factory, text_transform_factory

In [6]:
configs = get_experiment_config()
vocabulary = vocabulary_factory(config={
        'vocab_path': configs['vocab_path'] if configs['vocab_path'] else DEFAULT_VOCAB_PATHS[configs['dataset']],
        'vocab_threshold': configs['vocab_threshold']
    })



usage: ipykernel_launcher.py [-h] [--config_path CONFIG_PATH]
                             [--trainer TRAINER] [--epoch EPOCH]
                             [--evaluator EVALUATOR] [--vocab_path VOCAB_PATH]
                             [--vocab_threshold VOCAB_THRESHOLD]
                             [--dataset DATASET] [--batch_size BATCH_SIZE]
                             [--num_workers NUM_WORKERS] [--shuffle SHUFFLE]
                             [--use_transform USE_TRANSFORM]
                             [--img_size IMG_SIZE] [--metric_loss METRIC_LOSS]
                             [--gamma_scale GAMMA_SCALE]
                             [--feature_size FEATURE_SIZE]
                             [--text_feature_size TEXT_FEATURE_SIZE]
                             [--word_embedding_size WORD_EMBEDDING_SIZE]
                             [--image_encoder IMAGE_ENCODER] [--stride STRIDE]
                             [--text_encoder TEXT_ENCODER]
                             [--composito

SystemExit: 2

D:\IDE\Miniconda\envs\cv\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
models = create_models(configs, vocabulary)

NameError: name 'configs' is not defined